In [1]:
import sys
sys.path.insert(0, '../..')
from helper import _process_pred_labels
from evaluate import get_pred_labels

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [19]:
N_FILES = 2
WEIGHTS = (0.791782261928, 0.78721951357)
SUM_WEIGHTS = sum(WEIGHTS)

In [18]:
assert N_FILES == len(WEIGHTS)

In [20]:
pred_proba = pd.read_csv(f'sub_0/pred_proba.csv', index_col='review_id')
for col in pred_proba.columns:
    pred_proba[col] = 0

for i, w in zip(range(N_FILES), WEIGHTS):
    df = pd.read_csv(f'sub_{i}/pred_proba.csv', index_col='review_id')
    df = df.rank(ascending=False) / len(df)
    df *= (w / SUM_WEIGHTS)
    pred_proba += df

In [21]:
pred_proba

,0,1,2,3,4,5,6,7,8
review_id,,,,,,,,,
3,0.526566,0.842625,0.639174,0.151479,0.848505,0.764498,0.864743,0.594786,0.401404
4,0.437131,0.490752,0.490461,0.352449,0.552524,0.774460,0.518587,0.407907,0.446811
13,0.669290,0.905230,0.772948,0.620809,0.697467,0.554590,0.908914,0.541023,0.173699
14,0.214039,0.543390,0.534800,0.931115,0.510495,0.855922,0.629366,0.667859,0.763777
17,0.438847,0.871623,0.760888,0.752283,0.841765,0.783311,0.823013,0.785101,0.433435
...,...,...,...,...,...,...,...,...,...
192819,0.305801,0.286574,0.049701,0.170927,0.157498,0.279991,0.133370,0.133514,0.671754
192821,0.255733,0.600768,0.400417,0.604514,0.567188,0.701866,0.623109,0.435188,0.695974
192830,0.857713,0.043290,0.365142,0.720763,0.082103,0.010834,0.145514,0.288689,0.398499


In [24]:
blend_submition = pred_proba.copy()

In [15]:
pred_proba = pd.DataFrame(
    data=get_pred_labels(pred_proba.values),
    index=pred_proba.index
    columns=pred_proba.columns
)
blend_submition['target'] = pred_proba.apply(_process_pred_labels, axis=1)

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [ ]:
pred_proba

In [ ]:
pred_proba.to_csv('stacking3.csv')